# API Face Samples

## Objective
This script adds a face to a face collection (face list, large face list, person group, large person group, or person directory) using the Azure Face API. If more than one face is detected in the image, the largest face will be added.

In [ ]:
import os, requests, time
from PIL import Image
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel

def detect_faces(subscription_key, endpoint, image_path):
    with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=add-face-to-collection"}) as face_client:
        with open(image_path, 'rb') as image_data:
            detected_faces = face_client.detect(
                    image_content=image_data.read(),
                    detection_model=FaceDetectionModel.DETECTION_03,
                    recognition_model=FaceRecognitionModel.RECOGNITION_04,
                    return_face_id=True,
                )
        return detected_faces
    
def enlarge_bounding_box(face_rectangle, image_width, image_height, enlargement_factor=1.2):
    left = max(0, face_rectangle['left'] - (face_rectangle['width'] * (enlargement_factor - 1) / 2))
    top = max(0, face_rectangle['top'] - (face_rectangle['height'] * (enlargement_factor - 1) / 2))
    width = min(image_width - left, face_rectangle['width'] * enlargement_factor)
    height = min(image_height - top, face_rectangle['height'] * enlargement_factor)
    return {'left': int(left), 'top': int(top), 'width': int(width), 'height': int(height)}

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height
    
def check_operation_status(subscription_key, operation_location):
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'X-MS-AZSDK-Telemetry': 'sample=add-face-to-collection'
    }
    
    while True:
        response = requests.get(operation_location, headers=headers)
        if response.status_code == 200:
            status = response.json()
            if status.get('status') in ['succeeded', 'failed']:
                if status.get('status') == 'succeeded':
                    return "Face added successfully."
                else:
                    return f"Failed to add face: {status.get('message')}"
        else:
            print("Error checking operation status:", response.status_code, response.text)
            break

        time.sleep(5)  # Wait for a few seconds before checking again
    
def add_face(subscription_key, endpoint, image_path, target_type, target_id, person_id=None):
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Content-Type': 'application/octet-stream',
        'X-MS-AZSDK-Telemetry': 'sample=add-face-to-collection'
    }
    params = {
        'detectionModel': 'detection_03'
    }
        
    faces = detect_faces(subscription_key, endpoint, image_path)
    if len(faces) == 0:
        return "No faces detected in the image."
    elif len(faces) > 1:
        image_width, image_height = get_image_dimensions(image_path)
        # If multiple faces are detected, use the first face (largest face) for adding to the target
        print(f"Multiple faces detected. Using the first face (largest face) for adding to the target.")
        face_rectangle = enlarge_bounding_box(faces[0]['faceRectangle'], image_width, image_height)
        params['targetFace'] = f"{face_rectangle['left']},{face_rectangle['top']},{face_rectangle['width']},{face_rectangle['height']}"
    else:
        print(f"One face detected. Adding to the target.")

    if target_type == "face_list":
        # URL to add face to face list
        add_face_url = endpoint + f"/face/v1.0/faceLists/{target_id}/persistedFaces"
    elif target_type == "large_face_list":
        # URL to add face to large face list
        add_face_url = endpoint + f"/face/v1.0/largefacelists/{target_id}/persistedFaces"
    elif target_type == "person_group":
        if not person_id:
            return "Person ID is required for adding to a person group."
        # URL to add face to person group
        add_face_url = endpoint + f"/face/v1.0/persongroups/{target_id}/persons/{person_id}/persistedFaces"
    elif target_type == "large_person_group":
        if not person_id:
            return "Person ID is required for adding to a large person group."
        # URL to add face to large person group
        add_face_url = endpoint + f"/face/v1.0/largepersongroups/{target_id}/persons/{person_id}/persistedFaces"
    elif target_type == "person_directory":
        # URL to add face to person directory
        add_face_url = endpoint + f"/face/v1.0-preview/persons/{target_id}/recognitionModels/recognition_04/persistedFaces"
    else:
        return "Invalid target type specified."

    # Add the face to the specified target
    with open(image_path, 'rb') as image_data:
        response = requests.post(add_face_url, params=params, headers=headers, data=image_data)
        if response.status_code == 200:
            return "Face added successfully."
        elif response.status_code == 202:
            operation_location = response.headers.get('Operation-Location')
            if operation_location:
                return check_operation_status(subscription_key, operation_location)
            else:
                print("No Operation-Location header found in the response.")
        else:
            return f"Failed to add face: {response.json()}"

# Example usage
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'
target_type = 'face_list'  # or 'large_face_list', 'person_group', 'large_person_group', or 'person_directory'
target_id = 'your_face_list_id'  # or person group ID 
person_id = None  # Required if target_type is 'person_group' or 'large_person_group'


result = add_face(FACE_KEY, FACE_ENDPOINT, image_path, target_type, target_id, person_id)
print(result)